<img width="8%" alt="Naas Chat Plugin.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas%20Chat%20Plugin.png" style="border-radius: 15%">

# Naas Chat Plugin - Create prompt plugin with command
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Naas+Chat+Plugin+-+Create+prompt+plugin+with+command:+Error+short+description">Bug report</a>

**Tags:** #naaschatplugin #naas #naas_driver #chat #ai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-09-25 (Created: 2023-09-25)

**Description:** This notebook will provide step-by-step instructions on creating a prompt plugin with a specific command to be used in Naas Chat.

**References:**
- [Naas Chat Documentation](https://site.naas.ai/docs/platform/aI-powered-chat)
- [Naas Chat Plugin driver](https://github.com/jupyter-naas/drivers/blob/main/naas_drivers/tools/naas_chat_plugin.py)

## Input

### Import libraries

In [ ]:
from naas_drivers import naas_chat_plugin
from IPython.display import Markdown
import naas

### Setup variables
**Mandatory**
- `name`: The name of the plugin.
- `command_name`: The name of the command.
- `system_prompt`: The system prompt for the plugin.
- `first_number`: Variable to be used with your command. First number to be multiply.
- `second_number`: Variable to be used with your command. Second number to be multiply.

**Optional**
- `body`: This variable stores the body to be send by the webhook.
- `model`: The name of the model to be used for tokenization. Default is "gpt-3.5-turbo-16k".
- `temperature`: The temperature parameter for the model. Default is 0.
- `output_path`: The path where the JSON file should be saved. If not provided, it will be created from the plugin name.

In [ ]:
# Mandatory
name = "Calculator"
command_name = "Multiply_two_number"
system_prompt = f"""
You are simple calculator helping people to perform mathematical operation.
Start presenting yourself and tell the user they can do the slash command /{command_name}.
"""
first_number = 1
second_number = 2

# Optional
body = {}
model = "gpt-3.5-turbo-16k"
temperature = 0
output_path = None

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [ ]:
# Parameters
if len(body) > 0:
    first_number = float(body.get("first_number"))
    second_number = float(body.get("second_number"))

## Model

### Function to apply
In this case, multiply 2 numbers

In [ ]:
result = first_number * second_number
print(result)

### Create the message
Message to be sent in the input bar of your Naas Chat.

In [ ]:
message = f"{result}"

## Output

### JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
naas.webhook.respond_json(
    {
        "status": "ok", 
        "message": message
    }
)

### Push Webhook to production
Webhook URL to be included in command of your Naas Chat plugin.

In [ ]:
webhook_url = naas.webhook.add(params={"inline": True})

### Create command
Modify payload with parameters

In [ ]:
commands = [
    {
        "name": command_name,
        "action": {
          "request_type": "POST",
          "url": webhook_url,
          "payload": {
              "first_number": {
                  "type": "str",
                  "description": "First number",
                  "default": 1
              },
              "second_number": {
                  "type": "str",
                  "description": "Second number",
                  "default": 2
              },
          }
        }
    }
]

### Create Naas Chat plugin
This function will generate the plugin in JSON format and also verify if your prompt adheres to the recommended limit, which is set at 20% of the maximum tokens allowed by the model. Then, it will save your plugin in your local environment.

In [ ]:
plugin_file_path = naas_chat_plugin.create_plugin(
    name=name,
    prompt=system_prompt,
    model=model,
    temperature=temperature,
    output_path=output_path,
    commands=commands
)

### Create asset
This asset can be utilized by using the command `/use` in your Naas Chat or by simply clicking on the link provided in the cell below.

In [ ]:
plugin_url = naas.asset.add(plugin_file_path, params={"inline": True})

### Create new chat
You don't need to click on 'Create New Chat' everytime you update your system prompt, you can use the command `/refresh`.

In [ ]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")